In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
path = Path('data/midi')

In [5]:
import pandas as pd

In [6]:
source_dir = 'midi_sources_v3'
out_dir = 'midi_transform_v3'
source_csv = path/'metadata_v3'/f'{source_dir}.csv'
out_csv = path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)

In [8]:
source_csv

PosixPath('data/midi/metadata_v3/midi_sources_v3.csv')

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,10,11,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source,title,midi_title,inferred_offset,inferred_key,ht_offset,ht_mode,artist,seconds,bpm,...,instruments,time_signature,ht_time_signature,ht_bpm,song_url,midi,parts,genres,quarter_length,mxl
0,hooktheory,yu-gi-oh-theme-song,yu-gi-oh3,0.0,C major,0.0,1.0,wayne-sharpe,15.000000,128.0,...,"Piano,Piano",4/4,4.0,128.0,https://www.hooktheory.com/theorytab/view/wayn...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",NaN,NaN,NaN
1,hooktheory,beverly-hills,My New Song,NaN,NaN,0.0,NaN,weezer,NaN,NaN,...,NaN,NaN,4.0,128.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,intro-and-verse,NaN,NaN,NaN
2,hooktheory,falling-for-you,falling for you intro,-3.0,E- major,-3.0,1.0,weezer,11.111111,108.0,...,"Piano,Piano",4/4,4.0,108.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,verse,chorus,solo",NaN,NaN,NaN
3,hooktheory,buddy-holly,Buddy Holly,4.0,A- major,4.0,1.0,weezer,43.388430,121.0,...,"Piano,Piano",4/4,4.0,121.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"verse,pre-chorus,chorus,bridge,solo","Pop,Rock",NaN,NaN
4,hooktheory,dreams-of-grandeur,dreams of grandeur,-2.0,B minor,3.0,1.0,wavves,21.333333,180.0,...,"Piano,Piano",4/4,4.0,180.0,https://www.hooktheory.com/theorytab/view/wavv...,data/midi/midi_sources/hooktheory/pianoroll/w/...,verse,NaN,NaN,NaN


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(33240, 22)

In [9]:
from data_sources import process_parallel

In [10]:
from midi_transform import *

### Need better midi formatting

In [11]:
def transform_midi_from_row(idxrow):
    idx,row = idxrow
    midi_file = row.midi
    
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    
    offset = row.ht_offset if row.source == 'hooktheory' else row.inferred_offset
    try:
        transform_midi(midi_file, out_file, offset=offset)
    except Exception as e:
        print('Exception with midifile:', midi_file, e)
        return idx, None
    return idx, str(out_file)

In [12]:
# for r in df_filtered.iterrows():
#     transform_midi_from_row(r)

In [14]:
idx2out = process_parallel(transform_midi_from_row, df_filtered.iterrows(), total=df_filtered.shape[0])

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(idx2out.values())}, index=list(idx2out.keys()))

In [ ]:
merged_df = df.join(tdf)

In [ ]:
tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv, index=False); merged_df.head()